# **Download Libraries**

In [ ]:
!pip install tensorboardX
!pip install -q evaluate seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.

# **Import libraries**

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.utils as utils
import torchvision
import torch.optim as optim
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
from sklearn.metrics import accuracy_score
from sklearn import metrics
from datetime import datetime
from tensorboardX import SummaryWriter
import matplotlib.pyplot as plt
import cv2
import math
import os
import numpy as np
from tqdm.auto import tqdm
import evaluate
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


# **Build Attention ResNet**

## **Attention layers**

In [ ]:
class ProjectorBlock(nn.Module):
    def __init__(self, in_features, out_features):
        super(ProjectorBlock, self).__init__()
        self.op = nn.Conv2d(in_channels=in_features, out_channels=out_features,
            kernel_size=1, padding=0, bias=False)

    def forward(self, x):
        return self.op(x)

class SpatialAttn(nn.Module):
    def __init__(self, in_features, normalize_attn=True):
        super(SpatialAttn, self).__init__()
        self.normalize_attn = normalize_attn
        self.op = nn.Conv2d(in_channels=in_features, out_channels=1,
            kernel_size=1, padding=0, bias=False)

    def forward(self, l, g):
        N, C, H, W = l.size()
        c = self.op(l+g) # (batch_size,1,H,W)
        if self.normalize_attn:
            a = F.softmax(c.view(N,1,-1), dim=2).view(N,1,H,W)
        else:
            a = torch.sigmoid(c)
        g = torch.mul(a.expand_as(l), l)
        if self.normalize_attn:
            g = g.view(N,C,-1).sum(dim=2) # (batch_size,C)
        else:
            g = F.adaptive_avg_pool2d(g, (1,1)).view(N,C)
        return c.view(N,1,H,W), g

## **Residual block**

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride = 1, downsample = None):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Sequential(
                        nn.Conv2d(in_channels, out_channels, kernel_size = 3, stride = stride, padding = 1),
                        nn.BatchNorm2d(out_channels),
                        nn.ReLU())
        self.conv2 = nn.Sequential(
                        nn.Conv2d(out_channels, out_channels, kernel_size = 3, stride = 1, padding = 1),
                        nn.BatchNorm2d(out_channels))
        self.downsample = downsample
        self.relu = nn.ReLU()
        self.out_channels = out_channels

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.conv2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

## **Build the network**

In [ ]:
class AttnResNet(nn.Module):
    def __init__(self, sample_size, block, layers, num_classes, attention=True, normalize_attn=True):
        super(AttnResNet, self).__init__()
        # conv blocks
        self.inplanes = 64
        self.conv1 = nn.Sequential(
                        nn.Conv2d(3, 64, kernel_size = 7, stride = 2, padding = 3),
                        nn.BatchNorm2d(64),
                        nn.ReLU())
        self.maxpool = nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)
        self.layer0 = self._make_layer(block, 64, layers[0], stride = 1)
        self.layer1 = self._make_layer(block, 128, layers[1], stride = 2)
        self.layer2 = self._make_layer(block, 512, layers[2], stride = 2)
        self.layer3 = self._make_layer(block, 512, layers[3], stride = 2)
        self.avgpool = nn.AvgPool2d(5, stride=2)
        self.dense = nn.Conv2d(in_channels=512, out_channels=512, kernel_size= 1, padding=0, bias=True)
        # attention blocks
        self.attention = attention
        if self.attention:
            self.projector = ProjectorBlock(128, 512)
            self.attn1 = SpatialAttn(in_features=512, normalize_attn=normalize_attn)
            self.attn2 = SpatialAttn(in_features=512, normalize_attn=normalize_attn)
            self.attn3 = SpatialAttn(in_features=512, normalize_attn=normalize_attn)
        # final classification layer
        if self.attention:
            self.classify = nn.Linear(in_features=512*3, out_features=num_classes, bias=True)
        else:
            self.classify = nn.Linear(in_features=512, out_features=num_classes, bias=True)

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes, kernel_size=1, stride=stride),
                nn.BatchNorm2d(planes),
            )
        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool(x)

        x = self.layer0(x)

        l1 = self.layer1(x)

        l2 = self.layer2(l1)

        l3 = self.layer3(l2)

        x = self.avgpool(l3)

        #x = x.view(x.size(0), -1)

        g = self.dense(x) # batch_sizex512x1x1
        # attention
        if self.attention:
            c1, g1 = self.attn1(self.projector(l1), g)
            c2, g2 = self.attn2(l2, g)
            c3, g3 = self.attn3(l3, g)
            g = torch.cat((g1,g2,g3), dim=1) # batch_sizex3C
            # classification layer
            x = self.classify(g) # batch_sizexnum_classes
        else:
            c1, c2, c3 = None, None, None
            x = self.classify(torch.squeeze(g))
        return [x, c1, c2, c3]

## **Train / Val epochs**

In [ ]:
def train_epoch(model, criterion, optimizer, dataloader, device, epoch, log_interval, writer):
    model.train()
    losses = []
    all_label = []
    all_pred = []

    for batch_idx, (inputs, labels) in enumerate(dataloader):
        # get the inputs and labels
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        # forward
        outputs = model(inputs)
        if isinstance(outputs, list):
            outputs = outputs[0]

        # compute the loss
        loss = criterion(outputs, labels.squeeze())
        losses.append(loss.item())

        # compute the accuracy
        prediction = torch.max(outputs, 1)[1]
        all_label.extend(labels.squeeze())
        all_pred.extend(prediction)
        score = accuracy_score(labels.squeeze().cpu().data.squeeze().numpy(), prediction.cpu().data.squeeze().numpy())

        # backward & optimize
        loss.backward()
        optimizer.step()

        if (batch_idx + 1) % log_interval == 0:
            print("epoch {:3d} | iteration {:5d} | Loss {:.6f} | Acc {:.2f}%".format(epoch+1, batch_idx+1, loss.item(), score*100))
    # Compute the average loss & accuracy
    training_loss = sum(losses)/len(losses)
    all_label = torch.stack(all_label, dim=0)
    all_pred = torch.stack(all_pred, dim=0)
    training_acc = accuracy_score(all_label.squeeze().cpu().data.squeeze().numpy(), all_pred.cpu().data.squeeze().numpy())
    # Log
    writer.add_scalars('Loss', {'train': training_loss}, epoch+1)
    writer.add_scalars('Accuracy', {'train': training_acc}, epoch+1)
    print("Average Training Loss of Epoch {}: {:.6f} | Acc: {:.2f}%".format(epoch+1, training_loss, training_acc*100))


def val_epoch(model, criterion, dataloader, device, epoch, writer):
    model.eval()
    losses = []
    all_label = []
    all_pred = []

    with torch.no_grad():
        for batch_idx, (inputs, labels) in enumerate(dataloader):
            # get the inputs and labels
            inputs, labels = inputs.to(device), labels.to(device)
            # forward
            outputs = model(inputs)
            if isinstance(outputs, list):
                outputs = outputs[0]
            # compute the loss
            loss = criterion(outputs, labels.squeeze())
            losses.append(loss.item())
            # collect labels & prediction
            prediction = torch.max(outputs, 1)[1]
            all_label.extend(labels.squeeze())
            all_pred.extend(prediction)

    # Compute the average loss & accuracy
    val_loss = sum(losses)/len(losses)
    all_label = torch.stack(all_label, dim=0)
    all_pred = torch.stack(all_pred, dim=0)
    val_acc = accuracy_score(all_label.squeeze().cpu().data.squeeze().numpy(), all_pred.cpu().data.squeeze().numpy())
    # Log
    writer.add_scalars('Loss', {'val': val_loss}, epoch+1)
    writer.add_scalars('Accuracy', {'val': val_acc}, epoch+1)
    print("Average Validation Loss: {:.6f} | Acc: {:.2f}%".format(val_loss, val_acc*100))

# **Import dataset**

In [ ]:
#train and test data directory
data_dir = "/gdrive/MyDrive/data/train_resnet/train/"
test_data_dir = "/gdrive/MyDrive/data/train_resnet/test/"

#load the train and test data with augmentation
dataset = ImageFolder(data_dir,transform = transforms.Compose([
    transforms.Resize((150,150)),
    transforms.ToTensor(),

]))
test_dataset = ImageFolder(test_data_dir,transforms.Compose([
    transforms.Resize((150,150)),
    transforms.ToTensor()
]))

In [ ]:
batch_size = 7
val_size = 500
train_size = len(dataset) - val_size

train_data,val_data = random_split(dataset,[train_size,val_size])
print(f"Length of Train Data : {len(train_data)}")
print(f"Length of Validation Data : {len(val_data)}")

Length of Train Data : 5063
Length of Validation Data : 500


In [ ]:
#load the train and validation into batches.
train_loader = DataLoader(train_data, batch_size, shuffle = True, num_workers = 4, pin_memory = True)
val_loader = DataLoader(val_data, batch_size*2, num_workers = 4, pin_memory = True)
test_loader = DataLoader(test_dataset, batch_size*2, num_workers = 4, pin_memory = True)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
# switch to cuda
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

# **Let's train !**

In [ ]:
# Create model
model = AttnResNet(sample_size=32, block = ResidualBlock, layers = [3, 4, 6, 3], num_classes=3).to(device)
if torch.cuda.device_count() > 1:
    print("Using {} GPUs".format(torch.cuda.device_count()))
    model = nn.DataParallel(model)

In [ ]:
writer = SummaryWriter("runs/cnn_attention_{:%Y-%m-%d_%H-%M-%S}".format(datetime.now()))

In [ ]:
num_epochs = 20
lr = 1e-4
no_save = False
log_interval = 100
weight_decay = 1e-4
save_path = "/gdrive/MyDrive/saved_models/resNet/"

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)


for epoch in range(num_epochs):
    train_epoch(model, criterion, optimizer, train_loader, device, epoch, log_interval, writer)
    val_epoch(model, criterion, val_loader, device, epoch, writer)
    # adjust learning rate
    # scheduler.step()
    if not no_save:
        torch.save(model.state_dict(), os.path.join(save_path, "cnn_epoch{:03d}.pth".format(epoch+1)))
        print("Saving Model of Epoch {}".format(epoch+1))

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


epoch   1 | iteration   100 | Loss 0.771114 | Acc 71.43%
epoch   1 | iteration   200 | Loss 0.864530 | Acc 42.86%
epoch   1 | iteration   300 | Loss 0.089553 | Acc 100.00%
epoch   1 | iteration   400 | Loss 0.107847 | Acc 100.00%
epoch   1 | iteration   500 | Loss 0.584193 | Acc 71.43%
epoch   1 | iteration   600 | Loss 0.305859 | Acc 71.43%
epoch   1 | iteration   700 | Loss 0.055511 | Acc 100.00%
Average Training Loss of Epoch 1: 0.526812 | Acc: 79.08%


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Average Validation Loss: 0.272510 | Acc: 88.60%
Saving Model of Epoch 1


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


...

Average Validation Loss: 0.084155 | Acc: 97.40%
Saving Model of Epoch 9


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


epoch  10 | iteration   100 | Loss 0.465709 | Acc 85.71%
epoch  10 | iteration   200 | Loss 0.020143 | Acc 100.00%
epoch  10 | iteration   300 | Loss 0.007004 | Acc 100.00%
epoch  10 | iteration   400 | Loss 0.000446 | Acc 100.00%
epoch  10 | iteration   500 | Loss 0.003053 | Acc 100.00%
epoch  10 | iteration   600 | Loss 0.006609 | Acc 100.00%
epoch  10 | iteration   700 | Loss 0.011141 | Acc 100.00%
Average Training Loss of Epoch 10: 0.076892 | Acc: 97.29%


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Average Validation Loss: 0.351149 | Acc: 89.60%
Saving Model of Epoch 10


## **Save the model**

In [ ]:
torch.save(model.state_dict(), "/gdrive/MyDrive/saved_models/resNet/resNet_model_state.pth")

In [ ]:
modelB = AttnResNet(sample_size=32, block = ResidualBlock, layers = [3, 4, 6, 3], num_classes=3).to(device)
modelB.load_state_dict(torch.load("/gdrive/MyDrive/saved_models/resNet/resNet_model_state.pth"), strict=False)

<All keys matched successfully>

# **Evaluation**

In [ ]:
criterion = nn.CrossEntropyLoss()
num_epochs = 10
lr = 1e-4
no_save = False
log_interval = 100
weight_decay = 1e-4
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
writer = SummaryWriter("runs/cnn_attention_{:%Y-%m-%d_%H-%M-%S}".format(datetime.now()))

In [ ]:
def evaluate_model(model, criterion, dataloader, device,  writer):
    model.eval()
    losses = []

    all_label = []
    all_pred = []

    with torch.no_grad():
        for batch_idx, (inputs, labels) in enumerate(dataloader):
            # get the inputs and labels
            inputs, labels = inputs.to(device), labels.to(device)
            # forward
            outputs = model(inputs)
            if isinstance(outputs, list):
                outputs = outputs[0]
            # compute the loss
            try :

              loss = criterion(outputs, labels.squeeze())
              losses.append(loss.item())
              # collect labels & prediction
              prediction = torch.max(outputs, 1)[1]
              all_label.extend(labels.squeeze())
              all_pred.extend(prediction)
            except :
              pass

    # Compute the average loss & accuracy
    val_loss = sum(losses)/len(losses)
    all_label = torch.stack(all_label, dim=0)
    all_pred = torch.stack(all_pred, dim=0)
    #val_acc = accuracy_score(all_label.squeeze().cpu().data.squeeze().numpy(), all_pred.cpu().data.squeeze().numpy())
    # Log
    #writer.add_scalars('Loss', {'val': val_loss})
    #writer.add_scalars('Accuracy', {'val': val_acc})
    #print("Average Validation Loss: {:.6f} | Acc: {:.2f}%".format(val_loss, val_acc*100))

    # Classification report
    print('\n\n\t\tCLASSIFICATIION METRICS\n')
    print(metrics.classification_report(all_label.squeeze().cpu().data.squeeze().numpy(), all_pred.cpu().data.squeeze().numpy(),
                                        target_names = ["handwiriting", "other", "table"]))
    return metrics.classification_report(all_label.squeeze().cpu().data.squeeze().numpy(), all_pred.cpu().data.squeeze().numpy(),
                                        target_names = ["handwiriting", "other", "table"], output_dict=True)

In [ ]:
report = evaluate_model(model, criterion, test_loader, device, writer)
df = pd.DataFrame(report).transpose()

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(




		CLASSIFICATIION METRICS

              precision    recall  f1-score   support

handwiriting       1.00      0.95      0.97       424
       other       0.80      1.00      0.89       465
       table       0.99      0.81      0.89       502

    accuracy                           0.91      1391
   macro avg       0.93      0.92      0.92      1391
weighted avg       0.93      0.91      0.91      1391



In [ ]:
print('\t\tCLASSIFICATIION METRICS\n\tCNN with attention mechanism for pipeline ')
print('______________________________________________________')
df

		CLASSIFICATIION METRICS
	CNN with attention mechanism for pipeline 
______________________________________________________


,precision,recall,f1-score,support
handwiriting,1.000000,0.990566,0.995261,424.000000
other,0.965293,0.956989,0.961123,465.000000
table,0.954813,0.970060,0.962376,501.000000
accuracy,0.971942,0.971942,0.971942,0.971942
macro avg,0.973369,0.972538,0.972920,1390.000000
weighted avg,0.972103,0.971942,0.971988,1390.000000


# **Inference**

In [ ]:
def to_device(data, device):
    "Move data to the device"
    if isinstance(data,(list,tuple)):
        return [to_device(x,device) for x in data]
    return data.to(device,non_blocking = True)

In [ ]:
def predict_img_class(img,model):
    """ Predict the class of image and Return Predicted Class"""
    img = to_device(img.unsqueeze(0), device)
    outputs =  model(img)[0]
    prediction = torch.max(outputs, 1)[1]
    """print(prediction)
    _, preds = torch.max(prediction, dim = 1)"""
    return dataset.classes[prediction[0].item()]

In [ ]:
from PIL import Image
import numpy as np
#open image file
img = Image.open("/gdrive/MyDrive/test_images/handwriting5.jpg")

#resize image
img = img.resize((150,150))

channels = transforms.ToTensor()(img)
if channels.shape[0]>3 or channels.shape[0]<3:
  img = img.convert(mode='RGB')

#convert image to tensor
img = transforms.ToTensor()(img)

#print image
plt.imshow(img.permute(1,2,0))

#prdict image label
print(f"Predicted Class : {predict_img_class(img,modelB)}")

FileNotFoundError: ignored